## Behaviour illustration

In this notebook, we plot trajectories related to navground simulations on two scenarios: the corridor and the finite cross. We depict such trajectories considering various behaviors.

We start by importing some relevant libraries as well as creating some necessary folders.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

import perdiver.perdiver as perdiver
from perdiver.navground_io import parser, run_navground

from navground import core, sim

plots_dir = os.path.join("plots", "visualisation")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

In [ ]:
Next, we run Navground simulations 

In [ ]:
args = parser.parse_args([
        '--scenario', 'Corridor',
        '--length', '8.0',
        '--width', '1.0',
        '--time_step', '0.1',
        '--radius', '0.08',
        '--num_agents', '11',
        '--max_speed', '0.166',
        '--num_steps', '400',
    ])


behavior_list = ["ORCA", "HL", "HRVO", "Dummy"]
marker_behavior = {"ORCA": "o", "HL": "X", "HRVO": "+", "Dummy": "*"}

# Runs as dictionary with behaviors
runs = {}
for behavior in behavior_list:
    args.behavior = behavior
    print(args.behavior)
    runs[behavior] = run_navground(args)
# end for

Now, we illustrate movements for each behavior in a single image.

In [ ]:
fig, ax = plt.subplots(nrows=len(behavior_list), figsize=(10, 1.5*len(behavior_list)))
timestep_list = list(range(100,200,10))
for i, behavior in enumerate(behavior_list):
    run = runs[behavior][0]
    perdiver.plot_timesteps_corridor(run, timestep_list, args.length, args.width, ax[i])
    ax[i].set_title(behavior, fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(plots_dir, "behaviors_simulations.png"))

In [ ]:
def plot_image_cross(run, image, timestep_list, side, ax):
    # Load poses and number of agents
    poses = run.poses
    num_agents = poses.shape[1]
    # Plot image and adjust bounds
    ax.imshow(image, extent=[-side/2, side/2, -side/2, side/2])
    ax.set_xlim([-side/2, side/2])
    ax.set_ylim([-side/2, side/2])
    # Plot trajectories
    for i in range(num_agents):
        for f, t in zip(timestep_list[:-1], timestep_list[1:]):
            edge = np.array([poses[f,i], poses[t,i]])
            ax.plot(edge[:,0], edge[:,1], color=mpl.colormaps['Set1'](i/(num_agents+3)))
        # end for
    # end for

In [ ]:
args.scenario = "Cross"
args.side = 3

# Runs as dictionary with behaviors
runs = {}
for behavior in behavior_list:
    args.behavior = behavior
    print(args.behavior)
    runs[behavior] = run_navground(args)
# end for

In [ ]:
fig, ax = plt.subplots(ncols=len(behavior_list), figsize=(4*len(behavior_list),4))
timestep_list = list(range(60,140,15))
for i, behaviour in enumerate(behavior_list):
    run = runs[behaviour][0]
    image = perdiver.image_run_timestep_list(run, timestep_list)
    plot_image_cross(run, image, timestep_list, args.side, ax[i])
    ax[i].set_title(behaviour, fontsize=20)

plt.savefig(os.path.join(plots_dir, "behaviours_cross.png"))